<a href="https://colab.research.google.com/github/celinka12/Data-Mining/blob/main/%5BQuestion%5D_Exercise_Week11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

!pip install mlxtend==0.23.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [97]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [98]:
unique_items = set()
for col in df.columns:
    unique_items.update(df[col].unique())

print(unique_items)


{nan, 'Bagel', 'Wine', 'Bread', 'Pencil', 'Milk', 'Eggs', 'Cheese', 'Diaper', 'Meat'}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [99]:
#create an itemset based on the products
products = set()
for col in df.columns:
    products.update(df[col].unique())

encoded_transactions = []
for _, row in df.iterrows():
    transaction_dict = {product: (1 if product in row.values else 0) for product in products}
    encoded_transactions.append(transaction_dict)

# encoding the feature

encoded_transactions[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{nan: 0,
 'Bagel': 0,
 'Wine': 1,
 'Bread': 1,
 'Pencil': 1,
 'Milk': 0,
 'Eggs': 1,
 'Cheese': 1,
 'Diaper': 1,
 'Meat': 1}

In [100]:
# create new dataframe from the encoded features
# Replace NaN with a specific value, e.g., 'No Product'
df_with_nan = df.fillna('NaN')

# Flatten the data again, but keep 'No Product' as a category
flattened_data = df_with_nan.values.flatten()

# Step 1: Use OneHotEncoder, including 'No Product' (which represents missing values)
# Replace 'sparse' with 'sparse_output'
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # Make sure unknown values are ignored
encoded_data = encoder.fit_transform(flattened_data.reshape(-1, 1))

# Step 2: Create the one-hot encoded DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.categories_[0])

# Step 3: Create a DataFrame with 0s and 1s, representing whether the item is bought
product_data = pd.DataFrame(0, index=df.index, columns=encoder.categories_[0])

# Loop through each row and mark presence of products, including 'No Product'
for i, row in df_with_nan.iterrows():
    for product in row:
        product_data.loc[i, product] = 1

# Display the final DataFrame

# show the new dataframe
# print("Final one-hot encoded dataset with 'No Product':")
product_data.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,NaN,Pencil,Wine
0,0,1,1,1,1,1,0,0,1,1
1,0,1,1,1,0,1,1,0,1,1
2,0,0,1,0,1,1,1,1,0,1
3,0,0,1,0,1,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1,1


In [101]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
if 'NaN' in product_data.columns:
    product_data.drop(columns=['NaN'], inplace=True)

product_data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,0,1,1,1,1,1,0,1,1
1,0,1,1,1,0,1,1,1,1
2,0,0,1,0,1,1,1,0,1
3,0,0,1,0,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [102]:
#Set threshold value untuk digunakan dalam penghitungan support
min_support = 0.2

from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(product_data, min_support=min_support, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.425397,(Bagel)
1,0.504762,(Bread)
2,0.501587,(Cheese)
3,0.406349,(Diaper)
4,0.438095,(Eggs)
5,0.476190,(Meat)
6,0.501587,(Milk)
7,0.361905,(Pencil)
8,0.438095,(Wine)
9,0.279365,"(Bagel, Bread)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [104]:
threshold = 0.6
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=threshold)
rules.drop(columns=['zhangs_metric'], inplace=True)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
2,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
3,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
4,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
5,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
6,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
8,"(Cheese, Eggs)",(Meat),0.298413,0.476190,0.215873,0.723404,1.519149,0.073772,1.893773
9,"(Cheese, Meat)",(Eggs),0.323810,0.438095,0.215873,0.666667,1.521739,0.074014,1.685714


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

Association rule metrics help uncover patterns in transactional data. A
 data mining method used to find relationships or patterns between items in a dataset. These patterns are formulated in the form of rules with indicators such as support, confidence, and lift, which are used to measure the strength and relationships between items in a dataset.


 **Antecedent Support** represents the proportion of transactions containing the antecedent (left-hand side of the rule). For example, `(Bagel)` has an antecedent support of **0.425397**, meaning 42.54% of transactions include Bagel.

Similarly, **Consequent Support** measures the proportion of transactions containing the consequent (right-hand side of the rule). For instance, `(Bread)` has a consequent support of **0.504762**, meaning 50.48% of transactions include Bread.

**Support** calculates the proportion of transactions containing both antecedent and consequent, reflecting their co-occurrence. Using `(Bagel → Bread)` as an example, the support is **0.279365**, meaning 27.94% of transactions include both items.

**Confidence** quantifies the likelihood that the consequent occurs when the antecedent is present, showing the rule’s reliability. For `(Bagel → Bread)`, the confidence is **0.656716**, meaning 65.67% of transactions with Bagel also include Bread.

**Lift** measures the strength of the rule by comparing the observed co-occurrence of antecedent and consequent with their expected co-occurrence under independence. A lift greater than 1 indicates a positive association, as seen in `(Bagel → Bread)`, which has a lift of **1.301042**, suggesting these items occur together slightly more often than expected.

**Leverage** measures the difference between the observed and expected co-occurrence of antecedent and consequent. For `(Bagel → Bread)`, the leverage is **0.064641**, indicating a small positive deviation from what would be expected under independence.

Finally, **Conviction** reflects the dependency of the rule, indicating how often the antecedent occurs without the consequent. A higher conviction implies a stronger rule. For `(Bagel → Bread)`, the conviction is **1.442650**, meaning the rule is fairly strong but not absolute. Together, these metrics provide valuable insights into item relationships, enabling informed decisions in areas like market basket analysis.


# Interpretation of Metrics for Example Rules
Rule: Bagel → Bread:

**Antecedent Support**: 42.54% of transactions include Bagel.
**Consequent Support**: 50.48% of transactions include Bread.
**Confidence**: 65.67% of transactions with Bagel also have Bread.
**Lift**: 1.30 indicates Bagel and Bread occur together slightly more often than expected under independence.
Leverage: 0.0646 shows a small positive deviation from expected co-occurrence.
**Conviction**: 1.44 indicates moderate dependency of Bread on Bagel.

Rule: Eggs → Cheese:
**Confidence**: 68.12%, showing a stronger relationship between Eggs and Cheese.
**Lift**: 1.36 indicates that transactions with Eggs are 36% more likely to also have Cheese.
**Conviction**: 1.56 reflects a stronger dependence compared to Bagel → Bread.

Rule: Cheese, Eggs → Meat:
**Confidence**: 72.34%, meaning Cheese and Eggs together strongly predict Meat.
**Lift**: 1.52 shows a high level of positive association.
**Conviction**: 1.89 indicates a significant dependency, making this rule more reliable than the simpler rules above.


Referensi : Fikri, M. R., & Syahfitri, D. (2022). Penerapan metode association rule untuk analisis pola pembelian pelanggan di sebuah supermarket. Neliti. Diakses dari https://media.neliti.com/media/publications/554153-penerapan-metode-association-rule-untuk-45f642a0.pdf.
